In [58]:
!wget https://dphi-live.s3.eu-west-1.amazonaws.com/dataset/butterflies.zip

--2022-04-24 07:04:48--  https://dphi-live.s3.eu-west-1.amazonaws.com/dataset/butterflies.zip
Resolving dphi-live.s3.eu-west-1.amazonaws.com (dphi-live.s3.eu-west-1.amazonaws.com)... 52.218.20.3
Connecting to dphi-live.s3.eu-west-1.amazonaws.com (dphi-live.s3.eu-west-1.amazonaws.com)|52.218.20.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 239812069 (229M) [application/zip]
Saving to: ‘butterflies.zip.1’

butterflies.zip.1   100%[===================>] 228.70M  27.0MB/s    in 10s     

2022-04-24 07:04:59 (22.7 MB/s) - ‘butterflies.zip.1’ saved [239812069/239812069]



In [59]:
!apt-get install p7zip-full

Reading package lists... Done
Building dependency tree       
Reading state information... Done
p7zip-full is already the newest version (16.02+dfsg-6).
0 upgraded, 0 newly installed, 0 to remove and 40 not upgraded.


In [ ]:
!7za x /content/butterflies.zip


7-Zip (a) [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan /content/                   1 file, 239812069 bytes (229 MiB)

Extracting archive: /content/butterflies.zip
 44% 4096 Open              --
Path = /content/butterflies.zip
Type = zip
Physical Size = 239812069

  0%    
Would you like to replace the existing file:
  Path:     ./Butterfly Identification/Testing_set.csv
  Size:     43479 bytes (43 KiB)
  Modified: 2022-04-18 05:19:08
with the file from archive:
  Path:     Butterfly Identification/Testing_set.csv
  Size:     43479 bytes (43 KiB)
  Modified: 2022-04-18 05:19:08
? (Y)es / (N)o / (A)lways / (S)kip all / A(u)to rename all / (Q)uit? 

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import os
import cv2
from sklearn.model_selection import train_test_split

In [ ]:
labels = pd.read_csv("/content/Butterfly Identification/Training_set.csv")
labels.head()

In [ ]:
labels.tail()

In [ ]:
file_paths = [[fname, '/content/Butterfly Identification/train/' + fname] for fname in labels['filename']]

In [ ]:
if len(labels) == len(file_paths): 
  print('Number of labels i.e. ', len(labels), 'matches the number of filenames i.e. ', len(file_paths)) 
else: 
  print('Number of labels does not match the number of filenames')

In [ ]:
images = pd.DataFrame(file_paths, columns=['filename', 'filepaths']) 
images.head()

In [ ]:
train_data = pd.merge(images, labels, how = 'inner', on = 'filename') 
train_data.head()

In [ ]:
from sklearn.preprocessing import LabelEncoder 
le = LabelEncoder() 
train_data['label'] = le.fit_transform(train_data['label'])

In [ ]:
data = [] # initialize an empty numpy array 
image_size = 100 # image size taken is 100 here. one can take other size too 
for i in range(len(train_data)):  
  img_array = cv2.imread(train_data['filepaths'][i], cv2.IMREAD_GRAYSCALE) 
  # converting the image to gray scale  
  new_img_array = cv2.resize(img_array, (image_size, image_size)) # resizing the image array 
  data.append([new_img_array, train_data['label'][i]])

In [ ]:
4# image pixels of a image 
data[5]

In [ ]:
np.random.shuffle(data)

In [ ]:
x = [] 
y = [] 
for image in data: 
 x.append(image[0]) 
 y.append(image[1])  
 
# converting x & y to numpy array as they are list 
x = np.array(x) 
y = np.array(y)

In [ ]:
np.unique(y, return_counts=True)

In [ ]:
x = x.reshape(-1, 100, 100, 1)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(x,y,test_size=0.3, random_state = 42)

In [ ]:
cnn = tf.keras.models.Sequential([ 
                                  tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(100, 100, 1)), 
                                  tf.keras.layers.MaxPooling2D((2, 2)),  
                                  
                                  tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'), 
                                  tf.keras.layers.MaxPooling2D((2, 2)),  
                                  
                                  # tf.keras.layers.Flatten(input_shape=(100, 100, 1)), 
                                  tf.keras.layers.Flatten(), 
                                  tf.keras.layers.Dense(64, activation='relu'), 
                                  tf.keras.layers.Dense(75, activation='softmax')
                                   ])

In [ ]:
cnn.compile(optimizer='adam', 
            loss='sparse_categorical_crossentropy', 
            metrics=['accuracy'])

In [ ]:
cnn.fit(X_train, y_train, epochs=2, batch_size=1)

In [ ]:
cnn.evaluate(X_val, y_val)

In [ ]:
test_image_order = pd.read_csv("/content/Butterfly Identification/Testing_set.csv") 
test_image_order.head()

In [ ]:
file_paths = [[fname, '/content/Butterfly Identification/test/' + fname] for fname in test_image_order['filename']]

In [ ]:
if len(test_image_order) == len(file_paths): 
  print('Number of image names i.e. ', len(test_image_order), 'matches the number of file paths i.e. ', len(file_paths)) 
else:
  print('Number of image names does not match the number of filepaths')

In [ ]:
test_images = pd.DataFrame(file_paths, columns=['filename', 'filepaths']) 
test_images.head()

In [ ]:
#test_data = pd.merge(test_images, labels, how = 'inner', on = 'filename') 
#test_data.head()

In [ ]:
#from sklearn.preprocessing import LabelEncoder 
#le = LabelEncoder() 
#test_data['label'] = le.fit_transform(test_data['label'])

In [ ]:
test_pixel_data = [] 
image_size = 100 

for i in range(len(test_images)): 

  img_array = cv2.imread(test_images['filepaths'][i], cv2.IMREAD_GRAYSCALE) 
  new_img_array = cv2.resize(img_array, (image_size, image_size)) 
  test_pixel_data.append(new_img_array)

In [ ]:
test_pixel_data = np.array(test_pixel_data)

In [ ]:
test_pixel_data = test_pixel_data.reshape(-1, 100, 100, 1)

In [ ]:
pred = cnn.predict(test_pixel_data)

In [ ]:
pred[0]

In [ ]:
prediction = [] 
for value in pred: 
  prediction.append(np.argmax(value))

In [ ]:
predictions = le.inverse_transform(prediction)

In [ ]:
res = pd.DataFrame({'filename': test_data['filename'], 'label': predictions}) # prediction is nothing but the final predictions of your model on input features of your new unseen test data 
res.to_csv("submission.csv", index = False)  

# To download the csv file locally 
from google.colab import files  
files.download('submission.csv')